In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
# model_name, folder_name = "train_gdemo_2/","train_gdemo/"
# model_name, folder_name = "gdemo_FINAL3/","train_gdemo/"
model_name, folder_name = "train_gdemo_2/","old_gerb/"
data_directory  = 'drive/MyDrive/Colab Notebooks/Polyamanita/data/'+folder_name
model_directory = 'drive/MyDrive/Colab Notebooks/Polyamanita/models/'+model_name+'savedmodel.h5'

print(f'{data_directory}\n{model_directory}')

drive/MyDrive/Colab Notebooks/Polyamanita/data/old_gerb/
drive/MyDrive/Colab Notebooks/Polyamanita/models/train_gdemo_2/savedmodel.h5


In [4]:
size = 512
batch_size = 16

In [5]:
!pip -q install tensorflow tensorflow-gpu opencv-python matplotlib keras

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras import layers
import matplotlib
import pathlib
import cv2
import imghdr
import time
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential, load_model # MAY NEED TO BE REPLACED W/ FUNCTIONAL API
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, AveragePooling2D
from keras.metrics import Precision, Recall, BinaryAccuracy
from PIL import Image

In [7]:
model = tf.keras.models.load_model(model_directory)

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_directory,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(size, size),
    batch_size=batch_size
)

class_names = train_ds.class_names

Found 12397 files belonging to 20 classes.
Using 9918 files for training.


In [9]:
print(class_names)

['Chlorophyllum_molybdites', 'Clathrus_columnatus', 'Coprinellus_disseminatus', 'Coprinopsis_lagopus', 'Cylindrobasidium_laeve', 'Ganoderma_pfeifferi', 'Hortiboletus_rubellus', 'Leucocoprinus_birnbaumii', 'Leucocoprinus_fragilissimus', 'Lycoperdon_marginatum', 'Mycena_galericulata', 'Omphalotus_subilludens', 'Phallus_rugulosus', 'Pholiota_adiposa', 'Pleurotus_pulmonarius', 'Pycnoporus_cinnabarinus', 'Scleroderma_citrinum', 'Trametes_lactinea', 'Tricholoma_scalpturatum', 'Xerocomellus_chrysenteron']


In [10]:
img_url = [
    
    ["Chlorophyllum_molybdites", "https://lh3.googleusercontent.com/-ofxeXPKPbaU/VeYvQnjMSSI/AAAAAAAABVQ/R_M77R-Vcw8/s640/blogger-image-292714056.jpg"],
    ["Chlorophyllum_molybdites", "https://www.mushroomexpert.com/images/kuo3/chlorophyllum_molybdites_01.jpg"],
    ["Chlorophyllum_molybdites", "https://s3.amazonaws.com/media.jungledragon.com/images/3125/90437_small.jpeg?AWSAccessKeyId=05GMT0V3GWVNE7GGM1R2&Expires=1681948810&Signature=V8YBEJTeQXhpQhgRn8HGcUO98sk%3D"],

    ["Clathrus_columnatus", "https://www.mushroomexpert.com/images/kuo7/clathrus_columnatus_01.jpg"],
    ["Clathrus_columnatus", "https://static.inaturalist.org/photos/31066676/large.jpg"],
    ["Clathrus_columnatus", "https://inaturalist-open-data.s3.amazonaws.com/photos/67014835/original.jpeg"],

    ["Coprinellus_disseminatus", "https://www.first-nature.com/fungi/images/psathyrellaceae/coprinellus-disseminatus3.jpg"],
    ["Coprinellus_disseminatus", "https://inaturalist-open-data.s3.amazonaws.com/photos/11133/original.jpg"],
    ["Coprinellus_disseminatus", "https://live.staticflickr.com/7439/13099719233_c9e1422726_b.jpg"],

    # dog ass
    ["Coprinopsis_lagopus", "https://ffsc.us/sites/all/modules/drupalcommons/php/timthumb.php?src=data/files/images/bestphoto/Coprinopsis-lagopus-gr.TerryWay.jpg&w=650"],
    ["Coprinopsis_lagopus", "https://inpn.mnhn.fr/photos/uploads/webtofs/inpn/4/127774.jpg"],
    ["Coprinopsis_lagopus", "https://www.mykoweb.com/CAF/photos/large/Coprinopsis_lagopus%28fs-01%29.jpg"],

    ["Cylindrobasidium_laeve","https://live.staticflickr.com/65535/49484537306_b3f48b4039_b.jpg"],
    ["Cylindrobasidium_laeve","https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2i5lvJBXSqveM-vaaE69DY2AhZId9kyAosQ&usqp=CAU"],
    ["Cylindrobasidium_laeve","https://live.staticflickr.com/7851/45859587244_bd3d8fae5e_b.jpg"],

    ["Ganoderma_pfeifferi", "https://www.first-nature.com/fungi/images/ganodermataceae/ganoderma-pfeifferi2.jpg"],
    ["Ganoderma_pfeifferi", "https://www.first-nature.com/fungi/images/ganodermataceae/ganoderma-pfeifferi1.jpg"],
    ["Ganoderma_pfeifferi", "https://c8.alamy.com/comp/DAYX2E/beeswax-bracket-fungus-ganoderma-pfeifferi-ganodermataceae-syn-ganoderma-DAYX2E.jpg"],

    # sus
    ["Hortiboletus_rubellus", "https://www.first-nature.com/fungi/images/boletaceae/hortiboletus-rubellus6.jpg"],
    ["Hortiboletus_rubellus", "https://www.mushroomexpert.com/images/bochte/xerocomellus_rubellus_08.jpg"],
    ["Hortiboletus_rubellus", "https://live.staticflickr.com/4439/36538109635_68491c4912_b.jpg"],

    #

    ["Mycena_galericulata","https://www.first-nature.com/fungi/images/mycenaceae/mycena-galericulata1.jpg"],
    ["Mycena_galericulata","https://www.mushroomexpert.com/images/kuo7/mycena_galericulata_03.jpg"],
    ["Mycena_galericulata","https://www.first-nature.com/fungi/images/mycenaceae/mycena-galericulata1.jpg"],

    # ???
    ["Trametes_versicolor","https://upload.wikimedia.org/wikipedia/commons/a/a3/Trametes_versicolor_G4_%281%29.JPG"],
    # ["Trametes_versicolor","https://www.mushroom-appreciation.com/wp-content/uploads/2022/05/4-turkey-tail-mushrooms.jpg"],
    ["Trametes_versicolor","https://www.messiah.edu/Oakes/fungi_on_wood/poroid%20fungi/images/Tra%20versicolor%20Nathan%20Wilson.jpg"],

    ["Tricholoma_scalpturatum","https://upload.wikimedia.org/wikipedia/commons/7/78/Tricholoma_scalpuratum_20061014wa.jpg"],
    ["Tricholoma_scalpturatum","https://c8.alamy.com/zooms/9/1ac2cda5571c480ca5441eab637130cd/2dk86n8.jpg"],
    ["Tricholoma_scalpturatum","https://www.hlasek.com/foto/tricholoma_scalpturatum_be4903.jpg"],

    ["Xerocomellus_chrysenteron","https://www.mycodb.fr/photos/Xerocomellus_chrysenteron_2014_rp_1.jpg"],
    ["Xerocomellus_chrysenteron","https://www.mykoweb.com/CAF/photos/large/Xerocomellus_diffractus_mgw-02.jpg"],
    ["Xerocomellus_chrysenteron","https://www.mushroomexpert.com/images/kuo5/xerocomellus_chrysenteron_02.jpg"],

]

In [11]:
count = 0
for i in img_url:
    count = count + 1
    try: 

        img_path = tf.keras.utils.get_file(origin=i[1])
        img = tf.keras.preprocessing.image.load_img(
            img_path, target_size=(size, size, 3)
        )

        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create a batch
        predictions = model.predict(img_array)

        # predictions_sigmoid = tf.nn.sigmoid(predictions)
        score = tf.nn.softmax(predictions[0])
        guessed_name = class_names[np.argmax(score)]
        percent = 100 * np.max(score)
        
        real_name = i[0]
        # print(
        #     f"Guessed: {guessed_name}\n%Confidence: {percent}.\nReal: {real_name}\n{guessed_name == real_name}\n"
        # )

        if guessed_name != real_name:
          print(
            f"Guessed: {guessed_name}\n\t %Confidence: {percent}.\n\t Real: {real_name}"
          )
        
    except Exception as e:
        print(f'Bad: Image No. {count}')
        continue

In [12]:
saved_model_dir = model_directory = 'drive/MyDrive/Colab Notebooks/Polyamanita/models/'+model_name+'/'
os.listdir(saved_model_dir)

['savedmodel',
 'savedmodel.h5',
 'mushroom.tflite',
 'label_file.txt.gdoc',
 'pred_temp.npy',
 'class_analysis.csv',
 'Copy of test1.jpg',
 'Copy of test2.jpg',
 'test1.jpg',
 'test2.jpg']

In [13]:
!pip -q install pillow
import PIL

In [14]:
# 'Pholiota_adiposa'
# saved_model_dir + 'test.jpg'

In [15]:
# img = Image.open(saved_model_dir + 'test1.jpg')
# img = img.resize((size, size))
# img = np.array(img) / 255.0
# img = np.expand_dims(img, axis=0) # Add a batch dimension

# # Make predictions
# probs = model.predict(img)[0] # Get the probability distribution for the single image
# pred_class = np.argmax(probs) # Get the index of the highest probability

# # Get the predicted class name
# class_names = train_ds.class_names
# pred_class_name = class_names[pred_class]

# print("Predicted class:", pred_class_name)

In [16]:
img = Image.open(saved_model_dir + 'test1.jpg')
img = img.resize((size, size))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0) # Add a batch dimension

# Make predictions
probs = model.predict(img)[0] # Get the probability distribution for the single image
softmax_probs = tf.nn.softmax(probs) # Calculate the softmax probabilities

# Get the predicted class name and confidence score
class_names = train_ds.class_names
pred_class_index = tf.argmax(softmax_probs, axis=0) # Get the index of the highest probability
pred_class_name = class_names[pred_class_index]
confidence_score = softmax_probs[pred_class_index]

print("Predicted class:", pred_class_name)
print("Confidence score: {:.1f}%".format(confidence_score * 100))

1/1 [==============================] - 3s 3s/step
Predicted class: Mycena_galericulata
Confidence score: 98.8%


In [17]:
img = Image.open(saved_model_dir + 'test2.jpg')
img = img.resize((size, size))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0) # Add a batch dimension

# Make predictions
probs = model.predict(img)[0] # Get the probability distribution for the single image
softmax_probs = tf.nn.softmax(probs) # Calculate the softmax probabilities

# Get the predicted class name and confidence score
class_names = train_ds.class_names
pred_class_index = tf.argmax(softmax_probs, axis=0) # Get the index of the highest probability
pred_class_name = class_names[pred_class_index]
confidence_score = softmax_probs[pred_class_index]

print("Predicted class:", pred_class_name)
print("Confidence score: {:.1f}%".format(confidence_score * 100))

1/1 [==============================] - 0s 348ms/step
Predicted class: Mycena_galericulata
Confidence score: 98.3%


In [18]:
# img = Image.open(saved_model_dir + 'test3.jpg')
# img = img.resize((size, size))
# img = np.array(img) / 255.0
# img = np.expand_dims(img, axis=0) # Add a batch dimension

# # Make predictions
# probs = model.predict(img)[0] # Get the probability distribution for the single image
# softmax_probs = tf.nn.softmax(probs) # Calculate the softmax probabilities

# # Get the predicted class name and confidence score
# class_names = train_ds.class_names
# pred_class_index = tf.argmax(softmax_probs, axis=0) # Get the index of the highest probability
# pred_class_name = class_names[pred_class_index]
# confidence_score = softmax_probs[pred_class_index]

# print("Predicted class:", pred_class_name)
# print("Confidence score: {:.1f}%".format(confidence_score * 100))

In [19]:
# img = Image.open(saved_model_dir + 'test4.jpg')
# img = img.resize((size, size))
# img = np.array(img) / 255.0
# img = np.expand_dims(img, axis=0) # Add a batch dimension

# # Make predictions
# probs = model.predict(img)[0] # Get the probability distribution for the single image
# softmax_probs = tf.nn.softmax(probs) # Calculate the softmax probabilities

# # Get the predicted class name and confidence score
# class_names = train_ds.class_names
# pred_class_index = tf.argmax(softmax_probs, axis=0) # Get the index of the highest probability
# pred_class_name = class_names[pred_class_index]
# confidence_score = softmax_probs[pred_class_index]

# print("Predicted class:", pred_class_name)
# print("Confidence score: {:.1f}%".format(confidence_score * 100))